In [1]:
!git clone https://github.com/timothycao/agnews-classifier.git
%cd agnews-classifier

Cloning into 'agnews-classifier'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 37 (delta 17), reused 28 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 10.68 KiB | 3.56 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/kaggle/working/agnews-classifier


In [2]:
# Model

from peft import LoraConfig
from model import create_lora_model

lora_config = LoraConfig(
    r=2,
    lora_alpha=6,
    lora_dropout=0.1,
    bias='none',
    target_modules=['query', 'key'],
    task_type='SEQ_CLS',
    use_rslora = False # regular LoRA: lora_alpha/r, rs-LoRA: lora_alpha/sqrt(r) -> normalization technique
)

model = create_lora_model(lora_config)

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 667,396 || all params: 125,316,104 || trainable%: 0.5326


In [3]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Mod

In [4]:
# Training

from transformers import TrainingArguments
from train import main as train

training_args = TrainingArguments(
    # Core training configs
    max_steps=2000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='adamw_torch',
    learning_rate=5e-5,
    lr_scheduler_type='linear',
    
    # Logging, evaluation, and checkpointing
    output_dir='/kaggle/working/saved_models',
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Starting training...


Step,Training Loss,Validation Loss,Accuracy
100,1.374500,1.342785,0.598437
200,1.303100,1.227373,0.871875
300,1.060200,0.785594,0.864062
400,0.587600,0.400323,0.889062
500,0.391700,0.337846,0.887500
600,0.360100,0.326827,0.896875
700,0.334600,0.324566,0.896875
800,0.357700,0.314877,0.895312
900,0.347800,0.317368,0.898438
1000,0.330400,0.315551,0.898438


In [5]:
# Inference

from inference import main as inference

data_path = '/kaggle/input/deep-learning-spring-2025-project-2/test_unlabelled.pkl'
checkpoint = '/kaggle/working/saved_models/checkpoint-2000'
output_dir = '/kaggle/working/saved_predictions'

inference(data_path, checkpoint, output_dir)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running inference...


100%|██████████| 250/250 [00:57<00:00,  4.31it/s]

Predictions saved to /kaggle/working/saved_predictions/predictions_checkpoint-2000.csv


In [12]:
import pandas as pd

csv_path = '/kaggle/working/saved_models/checkpoint-2000/processed_log_history.csv'

df = pd.read_csv(csv_path)
df

,Step,Train Loss,Test Loss,Train Acc,Test Acc,Learning Rate,Epochs,Loss Spread,Loss Ratio,Acc Spread,Acc Ratio
0,100,1.3558,1.245366,0.397500,0.810937,0.000048,0.013405,0.110434,1.088676,-0.413437,0.490173
1,200,0.6322,0.345979,0.835268,0.885938,0.000047,0.026810,0.286221,1.827279,-0.050670,0.942807
2,300,0.3550,0.338247,0.876786,0.892188,0.000045,0.040214,0.016753,1.049528,-0.015402,0.982737
3,400,0.3036,0.328299,0.898214,0.901563,0.000043,0.053619,-0.024699,0.924768,-0.003348,0.996286
4,500,0.3174,0.316743,0.895536,0.906250,0.000042,0.067024,0.000657,1.002074,-0.010714,0.988177
5,600,0.2914,0.323829,0.903125,0.896875,0.000040,0.080429,-0.032429,0.899859,0.006250,1.006969
6,700,0.3101,0.308364,0.899107,0.912500,0.000038,0.093834,0.001736,1.005630,-0.013393,0.985323
7,800,0.3165,0.300734,0.895536,0.907813,0.000037,0.107239,0.015766,1.052426,-0.012277,0.986477
8,900,0.2881,0.307563,0.905357,0.903125,0.000035,0.120643,-0.019463,0.936720,0.002232,1.002472
9,1000,0.2862,0.302891,0.906250,0.901563,0.000033,0.134048,-0.016691,0.944895,0.004687,1.005199
